In [ ]:
import os
import sys
from pathlib import Path

import gpytorch

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from botorch.test_functions import Rastrigin

In [ ]:
from baybe import Campaign
from baybe.objectives import SingleTargetObjective
from baybe.acquisition.acqfs import LogExpectedImprovement, PosteriorMean, ProbabilityOfImprovement, UpperConfidenceBound, qExpectedImprovement, qLogExpectedImprovement, qProbabilityOfImprovement, qUpperConfidenceBound
from baybe.parameters import NumericalDiscreteParameter
from baybe.recommenders import RandomRecommender, SequentialGreedyRecommender, TwoPhaseMetaRecommender
from baybe.searchspace import SearchSpace
from baybe.simulation import simulate_scenarios
from baybe.targets import NumericalTarget
from baybe.utils.botorch_wrapper import botorch_function_wrapper
from baybe.utils.plotting import create_example_plots

In [ ]:
import pandas as pd
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.gaussian_process.kernels import RBF
from sklearn.gaussian_process.kernels import *
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import metrics

from typing import Optional
from torch import Tensor
from baybe.searchspace import SearchSpace
from baybe.surrogates import register_custom_architecture
# from baybe.utils.dataframe import add_fake_results

In [ ]:
%matplotlib inline

In [ ]:
SMOKE_TEST = "SMOKE_TEST" in os.environ

In [ ]:
SMOKE_TEST

False

In [ ]:
N_MC_ITERATIONS = 5
N_DOE_ITERATIONS = 5
BATCH_SIZE = 10
POINTS_PER_DIM = 10

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

X = pd.read_csv('~/../VVIRAL/data/AKTA_extracts/X.csv')
Y = pd.read_csv('~/../VVIRAL/data/AKTA_extracts/Y.csv')


class GetDummies(BaseEstimator, TransformerMixin):
    def __init__(self, dummy_columns):
        self.columns = None
        self.dummy_columns = dummy_columns

    def fit(self, X, y=None):
        self.columns = pd.get_dummies(X, columns=self.dummy_columns).columns
        return self

    def transform(self, X):
        X_new = pd.get_dummies(X, columns=self.dummy_columns)
        # print(X_new)
        X_new = X_new.loc[:, ~X_new.columns.duplicated()]
        # print(X_new)
        return X_new.reindex(columns=self.columns, fill_value=0)

x_dums = ['serotype_AAV10',
       'serotype_AAV2', 'serotype_AAV3', 'serotype_AAV5', 'serotype_AAV6',
       'serotype_AAV8', 'serotype_AAV9', 'from_ELU', 'from_LFT', 'resin_AAVA1',
       'resin_AAVA2', 'resin_AAVA3', 'resin_AAVA4', 'resin_AAVA5',
       'resin_AAVA6', 'resin_AAVA7', 'resin_AAVA8', 'resin_AAVW1',
       'resin_AAVW2', 'resin_AAVW3', 'resin_AAVW4', 'resin_AAVX', 'resin_AVB']
x_nom = ['Pure', 'Elution pH', 'Wash pH', 'Equlibration pH',
       'Elution Conductivity', 'Wash Conductivity',
       'Equilibration Conductivity', 'System Flowrate Elution (CV/h)',
       'Sample Flowrate Elution (CV/h)', 'Sample Volume']
dums = pd.from_dummies(X[x_dums], sep='_')

new_x = pd.concat([X[x_nom], dums], axis=1)
dummies = GetDummies(dummy_columns=['serotype', 'from', 'resin'])
dummies.fit(new_x)

GetDummies(dummy_columns=['serotype', 'from', 'resin'])

In [ ]:
new_x.shape

(206, 13)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, Y['y_log'], test_size=0.2, random_state=42, shuffle=True)
kernel = Sum(DotProduct(sigma_0=0.01, sigma_0_bounds=(1e-6, 1e6)), RationalQuadratic(alpha=0.01, length_scale=0.01,length_scale_bounds=(1e-6, 1e6))) + Matern(length_scale=0.1, nu=1.5,length_scale_bounds=(1e-6, 1e6))

# kernel = RationalQuadratic(alpha=0.01, length_scale=0.01,length_scale_bounds=(1e-6, 1e6)) + Matern(length_scale=0.1, nu=1.5,length_scale_bounds=(1e-6, 1e6))
kernel = Exponentiation(kernel, exponent=1)
scaled_experiment = False

scaler = MinMaxScaler()
x_train_scaled = scaler.fit_transform(x_train.values)
x_test_scaled = scaler.transform(x_test.values)
y_train_scaled = scaler.fit_transform(y_train.values.reshape(-1, 1))
y_test_scaled = scaler.transform(y_test.values.reshape(-1, 1))

def get_metrics(y_test, prediction):
    test_mse = round(metrics.mean_squared_error(y_test, prediction), 2)
    test_mae = round(metrics.mean_absolute_error(y_test, prediction), 2)
    test_rmse = round(metrics.root_mean_squared_error(y_test, prediction), 2)
    test_mape = round(metrics.mean_absolute_percentage_error(y_test, prediction), 2)
    test_r2 = round(metrics.r2_score(y_test, prediction), 2)
    print(f'MSE : {round(metrics.mean_squared_error(y_test, prediction), 2)}')
    print(f'MAE : {round(metrics.mean_absolute_error(y_test, prediction), 2)}')
    print(f'RMSE : {round(metrics.root_mean_squared_error(y_test, prediction), 2)}')
    print(f'MAPE : {round(metrics.mean_absolute_percentage_error(y_test, prediction), 2)}')
    print(f'R2 : {round(metrics.r2_score(y_test, prediction), 2)}')
    return {'mse': test_mse, 'mae': test_mae, 'rmse': test_rmse, 'mape': test_mape, 'r2': test_r2}

if scaled_experiment:
    print('Scaled Experiment')
    gaussian_process = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=50)
    gaussian_process.fit(x_train_scaled, y_train_scaled)
    mean_prediction, std_prediction = gaussian_process.predict(x_test_scaled, return_std=True)
    print('Scaled Experiment - metrics with scaled')
    test_metrics_scaled = get_metrics(y_test_scaled, mean_prediction)
    print(f'Min : {np.min(y_train_scaled)} Max : {np.max(y_train_scaled)}, Mean {np.mean(y_train_scaled)}, STD: {np.std(y_train_scaled)}')
    print(f'Min : {np.min(y_test_scaled)} Max : {np.max(y_test_scaled)}, Mean {np.mean(y_test_scaled)}, STD: {np.std(y_test_scaled)}')

else:
    print('Normal Experiment')
    gaussian_process = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=50)
    gaussian_process.fit(x_train, y_train)
    mean_prediction, std_prediction = gaussian_process.predict(x_test, return_std=True)
    test_metrics_unscaled = get_metrics(y_test, mean_prediction)
print(f'Min : {np.min(y_train)} Max : {np.max(y_train)}, Mean {np.mean(y_train)}, STD: {np.std(y_train)}')

Normal Experiment
MSE : 1.29
MAE : 0.83
RMSE : 1.13
MAPE : 0.03
R2 : 0.92
Min : 16.0279410482975 Max : 33.06036674488299, Mean 28.022778472417397, STD: 3.297324545512073


/usr/local/lib/python3.11/dist-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter kernel__k1__k2__alpha is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(


In [ ]:
x_train.shape

(164, 33)

In [ ]:
class Prob:
    def __init__(self):
        self.dim = 13
        self.ref_point = torch.tensor([20.0], **tkwargs)
        self.num_objectives = 1

In [ ]:
DIMENSION = 13
TestFunctionClass = Prob

In [ ]:
sim_cols = ['Pure', 'Equlibration pH', 'Elution Conductivity', 'Sample Volume', 'serotype', 'from', 'resin', 'Elution pH', 'Wash pH', 'Elution Conductivity', 'Wash Conductivity', 'Sample Flowrate Elution (CV/h)', 'System Flowrate Elution (CV/h)']
global history
history = pd.DataFrame(columns=sim_cols)

In [ ]:
def change_history(temp_df, history):
    # global history
    history = pd.concat([history, temp_df])
    return None

In [ ]:
history = []
def get_value(*x):
    temp_df = pd.DataFrame(x).T
    temp_df.columns = sim_cols
    # pd.concat([history, temp_df])
    history.append(x)
    return gaussian_process.predict(dummies.transform(temp_df))

In [ ]:
from baybe.targets import NumericalTarget
from baybe.objectives import SingleTargetObjective

target = NumericalTarget(
    name="Total Capsids",
    mode="MAX",
)
objective = SingleTargetObjective(target=target)

In [ ]:
from baybe.parameters import (
    CategoricalParameter,
    NumericalDiscreteParameter,
    NumericalContinuousParameter,
)

parameters = [
    NumericalDiscreteParameter(
        name="Pure",
        values=[0, 1],
    ),
    NumericalContinuousParameter(
        name="Elution pH",
        bounds=(5, 9),
    ),
    NumericalContinuousParameter(
        name="Wash pH",
        bounds=(5, 9),
    ),
    NumericalDiscreteParameter(
        name="Equilibration pH",
        values=[7.0, 0],
    ),
    NumericalContinuousParameter(
        name="Elution Conductivity",
        bounds=(1, 15),
    ),
    NumericalContinuousParameter(
        name="Wash Conductivity",
        bounds=(1, 15),
    ),
    NumericalDiscreteParameter(
        name="Equilibration Conductivity",
        values=[2.5, 0],
    ),
    NumericalContinuousParameter(
        name="System Flowrate Elution (CV/h)",
        bounds=(19.05, 50),
    ),
    NumericalContinuousParameter(
        name="Sample Flowrate Elution (CV/h)",
        bounds=(19.05, 50),
    ),
    NumericalDiscreteParameter(
        name="Sample Volume",
        values=[5, 10, 15, 20, 25, 30],
    ),
    CategoricalParameter(
        name="serotype",
        values=['AAV10','AAV2','AAV3','AAV5','AAV6','AAV8','AAV9'],
        encoding="OHE",  # one-hot encoding of categories
    ),
    CategoricalParameter(
        name="from",
        values=['LFT', 'ELU'],
        encoding="OHE",  # one-hot encoding of categories
    ),
    CategoricalParameter(
        name="resin",
        values=['AAVA1', 'AAVA2', 'AAVA3', 'AAVA4', 'AAVA5', 'AAVA6', 'AAVA7',
       'AAVA8', 'AAVW1', 'AAVW2', 'AAVW3', 'AAVW4', 'AAVX', 'AVB'],
        encoding="OHE",  # one-hot encoding of categories
    )

]

In [ ]:
searchspace = SearchSpace.from_product(parameters=parameters)

In [ ]:
# find indices of parameter values that you dont want recommended
dont_recommend_idxs_1 = searchspace.discrete.exp_rep['serotype'] != 'AAV2'

# restrict recommendations via metadata
searchspace.discrete.metadata.loc[dont_recommend_idxs_1, 'dont_recommend'] = True

# find indices of parameter values that you dont want recommended
dont_recommend_idxs_2 = searchspace.discrete.exp_rep['from'] != 'ELU'

# restrict recommendations via metadata
searchspace.discrete.metadata.loc[dont_recommend_idxs_2, 'dont_recommend'] = True

# find indices of parameter values that you dont want recommended
dont_recommend_idxs_3 = searchspace.discrete.exp_rep['resin'] != 'AAVA3'

# restrict recommendations via metadata
searchspace.discrete.metadata.loc[dont_recommend_idxs_3, 'dont_recommend'] = True

# find indices of parameter values that you dont want recommended
dont_recommend_idxs_4 = searchspace.discrete.exp_rep['Pure'] != 0

# restrict recommendations via metadata
searchspace.discrete.metadata.loc[dont_recommend_idxs_4, 'dont_recommend'] = True

# find indices of parameter values that you dont want recommended
dont_recommend_idxs_5 = searchspace.discrete.exp_rep['Equilibration pH'] != 7.0

# restrict recommendations via metadata
searchspace.discrete.metadata.loc[dont_recommend_idxs_5, 'dont_recommend'] = True

# find indices of parameter values that you dont want recommended
dont_recommend_idxs_6 = searchspace.discrete.exp_rep['Equilibration Conductivity'] != 2.5

# restrict recommendations via metadata
searchspace.discrete.metadata.loc[dont_recommend_idxs_6, 'dont_recommend'] = True

In [ ]:
from baybe.surrogates.gaussian_process.core import GaussianProcessSurrogate

from baybe.kernels import RBFKernel, RQKernel, MaternKernel, AdditiveKernel
from baybe.surrogates.gaussian_process.kernel_factory import PlainKernelFactory
from baybe.surrogates.gaussian_process.core import GaussianProcessSurrogate
from baybe.kernels.base import Kernel
import torch

# kernel = Sum(DotProduct(sigma_0=0.01, sigma_0_bounds=(1e-6, 1e6)), RationalQuadratic(alpha=0.01, length_scale=0.01,length_scale_bounds=(1e-6, 1e6))) + Matern(length_scale=0.1, nu=1.5,length_scale_bounds=(1e-6, 1e6))
# kernel = Exponentiation(kernel, exponent=1)

class DotProductKernel(Kernel):
    def __init__(self, sigma=1.0):
        super().__init__()
        object.__setattr__(self, '_sigma', sigma)  # Use object.__setattr__ for immutability

    @property
    def sigma(self):
        return self._sigma

    def forward(self, X, Y):
        return self.sigma**2 + torch.mm(X, Y.T)

    def to_gpytorch(self, ard_num_dims=None, batch_shape=torch.Size([]), active_dims=None):
        # Map your custom kernel to an existing gpytorch kernel
        return gpytorch.kernels.LinearKernel(
            num_dims=ard_num_dims,
            batch_shape=batch_shape,
            active_dims=active_dims,
            variance_prior=None,
            bias_prior=None,
        )


# Create an instance of the DotProduct kernel with sigma
dot_product_kernel = DotProductKernel(sigma=0.01)

rq_kernel = RQKernel(lengthscale_initial_value=0.01)
matern_kernel = MaternKernel(lengthscale_initial_value=0.1, nu=1.5)


kernel = AdditiveKernel((dot_product_kernel, rq_kernel, matern_kernel))
# Wrap the RBF kernel in a PlainKernelFactory
kernel_factory = PlainKernelFactory(kernel=kernel)

# Initialize the GaussianProcessSurrogate with the RBF kernel
gp_surrogate = GaussianProcessSurrogate(kernel_or_factory=kernel_factory)


In [ ]:
seq_greedy_EI_campaign = Campaign(
    searchspace=searchspace,
    objective=objective,
    recommender = TwoPhaseMetaRecommender(
        initial_recommender=RandomRecommender(),
        recommender=SequentialGreedyRecommender(acquisition_function=qExpectedImprovement(), surrogate_model=gp_surrogate)
        )
)

seq_greedy_logEI_campaign = Campaign(

    searchspace=searchspace,
    objective=objective,
    recommender = TwoPhaseMetaRecommender(
        initial_recommender=RandomRecommender(),
        recommender=SequentialGreedyRecommender(acquisition_function=qLogExpectedImprovement(), surrogate_model=gp_surrogate)
        )

)

seq_greedy_PI_campaign = Campaign(
    searchspace=searchspace,
    objective=objective,
    recommender = TwoPhaseMetaRecommender(
        initial_recommender=RandomRecommender(),
        recommender=SequentialGreedyRecommender(acquisition_function=qProbabilityOfImprovement(), surrogate_model=gp_surrogate)
        )
)

seq_greedy_UCB_campaign = Campaign(
    searchspace=searchspace,
    objective=objective,
    recommender = TwoPhaseMetaRecommender(
        initial_recommender=RandomRecommender(),
        recommender=SequentialGreedyRecommender(acquisition_function=qUpperConfidenceBound(), surrogate_model=gp_surrogate)
        )
)

random_campaign = Campaign(
    searchspace=searchspace,
    recommender=RandomRecommender(),
    objective=objective,
)


In [ ]:
scenarios = {
    "Sequential greedy EI": seq_greedy_EI_campaign,
    "Random": random_campaign,
    "Sequential greedy logEI": seq_greedy_logEI_campaign,
    "Sequential greedy PI": seq_greedy_PI_campaign,
     "Sequential greedy UCB": seq_greedy_UCB_campaign,
}
results = simulate_scenarios(
    scenarios,
    # WRAPPED_FUNCTION,
    get_value,
    batch_size=BATCH_SIZE,
    n_doe_iterations=N_DOE_ITERATIONS,
    n_mc_iterations=N_MC_ITERATIONS,
)

In [ ]:
%matplotlib inline
path = Path(sys.path[0])
ax = sns.lineplot(
    data=results,
    marker="o",
    markersize=10,
    x="Num_Experiments",
    y="Total Capsids_CumBest",
    hue="Scenario",
)

ax.set_ylabel(r'$log_{e}{TC}$')
ax.set_xlabel(r'Number of Experiments')

create_example_plots(
    ax=ax,
    path=path,
    base_name="botorch_analytical",
)


/usr/local/lib/python3.11/dist-packages/baybe/utils/plotting.py:60: UserWarning: No themes for plotting found. A fallback theme is used.
  warnings.warn("No themes for plotting found. A fallback theme is used.")


In [ ]:
import pickle

with open('ax.pkl', 'wb') as f:
    pickle.dump(ax, f)

In [ ]:
results.to_csv('results_custom_surrogate.csv', index=False)

In [ ]:
history = pd.DataFrame(history, columns=sim_cols)
history.to_csv('history_custom_surrogate.csv', index=False)